## 各种import

In [1]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("..\examples"))

import utils; reload(utils)
from utils import *

from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop

Using Theano backend.


## 从Keras导入数据

In [2]:
from keras.datasets import mnist
(trn_data, trn_res), (test_data, test_res) = mnist.load_data()
(trn_data.shape, trn_res.shape, test_data.shape, test_res.shape)

((60000L, 28L, 28L), (60000L,), (10000L, 28L, 28L), (10000L,))

## 处理数据

### One-Hot-Encoding

In [3]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
trn_label = onehot(trn_res)
test_label = onehot(test_res)

print(trn_res[:5])
print(trn_label[:5])

[5 0 4 1 9]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


### 对图像加入新的维度
即灰度, 维度大小为1; 彩色图片则维度大小为3, 代表RGB, 3个通道

In [4]:
trn_data = np.expand_dims(trn_data, 1)
test_data = np.expand_dims(test_data, 1)
trn_data.shape

(60000L, 1L, 28L, 28L)

### 对输入图像进行正则化

In [5]:
input_mean = trn_data.mean().astype(np.float32)
input_std = trn_data.std().astype(np.float32)

def norm_input(x):
    return (x-input_mean)/input_std

## 1. 简单神经网络模型

In [31]:
simpleNN = Sequential()
simpleNN.add(Lambda(norm_input, input_shape=(1, 28, 28)))
simpleNN.add(Flatten())
simpleNN.add(Dense(392, activation='softmax'))
simpleNN.add(Dense(10, activation='softmax'))
simpleNN.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

C:\anaconda2\lib\site-packages\keras\layers\core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


### 1.1 直接训练

In [8]:
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 10s - loss: 1.2791 - acc: 0.8240 - val_loss: 0.6379 - val_acc: 0.8467


### 1.2 生成batch再训练

利用ImageDataGenerator(), 使用fit_generator而不是fit
注意fit_generator的文档中参数由于版本升级发生了变化

In [32]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

In [40]:
simpleNN.optimizer.lr = 0.1
simpleNN.fit_generator(batches, 1024, nb_epoch=1, validation_data=test_batches, nb_val_samples=512)

Epoch 1/1
1024/1024 [==============================] - 0s - loss: 2.1017 - acc: 0.7480 - val_loss: 2.0914 - val_acc: 0.7188


In [42]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit_generator(batches, 1024, nb_epoch=3, validation_data=test_batches, nb_val_samples=512)

Epoch 1/3
1024/1024 [==============================] - 0s - loss: 2.0490 - acc: 0.7783 - val_loss: 2.0396 - val_acc: 0.7637
Epoch 2/3
1024/1024 [==============================] - 0s - loss: 2.0340 - acc: 0.7529 - val_loss: 2.0083 - val_acc: 0.7988
Epoch 3/3
1024/1024 [==============================] - 0s - loss: 2.0121 - acc: 0.7393 - val_loss: 1.9958 - val_acc: 0.7812
